<a href="https://colab.research.google.com/github/Darshansingh11/SMAI-FP/blob/main/Deconvolution.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.models as models

In [2]:
alex_net = models.alexnet(pretrained=True)

/usr/local/lib/python3.7/dist-packages/torchvision/models/_utils.py:209: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  f"The parameter '{pretrained_param}' is deprecated since 0.13 and will be removed in 0.15, "
/usr/local/lib/python3.7/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=AlexNet_Weights.IMAGENET1K_V1`. You can also use `weights=AlexNet_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/alexnet-owt-7be5be79.pth" to /root/.cache/torch/hub/checkpoints/alexnet-owt-7be5be79.pth


  0%|          | 0.00/233M [00:00<?, ?B/s]

In [4]:
list(alex_net.features.children())[0].weight.shape

torch.Size([64, 3, 11, 11])

In [11]:
list(alex_net.features.children())[0].weight.permute(0,1,3,2).shape

torch.Size([64, 3, 11, 11])

In [22]:
list(alex_net.features.children())

[Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2)),
 ReLU(inplace=True),
 MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False),
 Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2)),
 ReLU(inplace=True),
 MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False),
 Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
 ReLU(inplace=True),
 Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
 ReLU(inplace=True),
 Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
 ReLU(inplace=True),
 MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)]

In [24]:
def get_layers(alex_net):
    #alex_net = models.alexnet(pretrained=True)
    layers = list(alex_net.features.children())

    layer1 = nn.Sequential(*layers[:2], nn.MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False, return_indices=True))
    layer2 = nn.Sequential(*layers[3:6])
    layer3 = nn.Sequential(*layers[6:8])
    layer4 = nn.Sequential(*layers[8:10])
    layer5 = nn.Sequential(*layers[10:13])

    conv_layers = [layer1, layer2, layer3, layer4, layer5]

    conv1_t = nn.ConvTranspose2d(64, 3, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
    conv2_t = nn.ConvTranspose2d(192, 64, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    conv3_t = nn.ConvTranspose2d(384, 192, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    conv4_t = nn.ConvTranspose2d(256, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    conv5_t = nn.ConvTranspose2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))

    conv_t_layers = [conv1_t, conv2_t, conv3_t, conv4_t, conv5_t]

    conv_i = [0,3,6,8,10]
    with torch.no_grad():
        for i,conv_t in enumerate(conv_t_layers):
            conv_t.weight = nn.Parameter(layers[conv_i[i]].weight.permute(0,1,3,2))
            conv_t.bias = nn.Parameter(layers[conv_i[i]].bias)

    return conv_layers, conv_t_layers

In [25]:
conv_layers, conv_t_layers = get_layers(alex_net)

In [21]:
img = torch.randn((1, 3, 256, 256))
img.shape

torch.Size([1, 3, 256, 256])

In [26]:
out, pos = conv_layers[0](img)
out.shape

torch.Size([1, 64, 31, 31])

In [27]:
pos.shape

torch.Size([1, 64, 31, 31])

In [28]:
out[:,1,:,:].shape

torch.Size([1, 31, 31])

In [31]:
unpool = nn.MaxUnpool2d(kernel_size=3, stride=2, padding=0)

In [32]:
o = unpool(out, pos)
o.shape

torch.Size([1, 64, 63, 63])

In [33]:
o = F.relu(o)

In [34]:
o = conv_t_layers[0](o)
o.shape

RuntimeError: ignored

In [35]:
nn.ConvTranspose2d(64, 3, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2)).bias.shape

torch.Size([3])